<a href="https://colab.research.google.com/github/Rachinder3/Text-Generation/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#get the data

!wget https://www.gutenberg.org/files/1661/1661-0.txt -o book.txt

In [2]:
text= open("/content/1661-0.txt",'r',encoding="utf-8").read()
text=text.lower()

In [3]:
# getting the sentences
sentences=text.split('\n')

In [4]:
# Loading the libraries
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [5]:
# Tokenizing the text

tokenizer=Tokenizer(oov_token='<UNK>')

In [6]:
tokenizer.fit_on_texts(sentences)

In [7]:
vocab_size= len(tokenizer.word_index)+ 1  # 1 takes care of oov token

In [8]:
vocab_size

8915

In [9]:
# converting words to sequences
sequences=tokenizer.texts_to_sequences(sentences)

In [10]:
# Building n gram sequences

input_sequences=[]

for sequence in sequences:
  for i in range(1,len(sequence)):
    n_gram_sequence=sequence[:i+1]
    input_sequences.append(n_gram_sequence)

In [11]:
input_sequences[0],input_sequences[1]

([4775, 145], [4775, 145, 132])

In [12]:
# padding the input sequences

# max sequence length
max_sequence_len=max([len(seq) for seq in input_sequences])

In [13]:
max_sequence_len

20

In [14]:
# padding
padded_sequence= pad_sequences(input_sequences,maxlen=max_sequence_len)


In [15]:
padded_sequence[0],padded_sequence[1]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0, 4775,  145], dtype=int32),
 array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0, 4775,  145,  132], dtype=int32))

# Creating the input output pairs

In [16]:
import numpy as np

In [17]:
padded_sequence=np.array(padded_sequence)

In [18]:
x=padded_sequence[:,:-1]

In [19]:
y=padded_sequence[:,-1]

# one hot encoding the targets

In [20]:
y=tf.keras.utils.to_categorical(y)

In [21]:
y.shape

(101575, 8915)

In [22]:
x.shape

(101575, 19)

#Building the model

In [23]:
# Loading the dependencies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam 

In [24]:
# defining and compiling the model

model=Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 21, 100)           891500    
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               731136    
_________________________________________________________________
dense (Dense)                (None, 8915)              4573395   
Total params: 6,196,031
Trainable params: 6,196,031
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Adding the call back
from tensorflow.keras.callbacks import EarlyStopping


In [27]:
es=EarlyStopping(monitor='accuracy',min_delta=0.01)

In [28]:
model.fit(x,y,epochs=50, verbose=True, batch_size=512)

Epoch 1/50
199/199 [==============================] - 19s 69ms/step - loss: 6.2055 - accuracy: 0.0851
Epoch 2/50
199/199 [==============================] - 14s 69ms/step - loss: 5.3375 - accuracy: 0.1411
Epoch 3/50
199/199 [==============================] - 14s 69ms/step - loss: 4.8120 - accuracy: 0.1696
Epoch 4/50
199/199 [==============================] - 14s 69ms/step - loss: 4.3045 - accuracy: 0.1998
Epoch 5/50
199/199 [==============================] - 14s 69ms/step - loss: 3.8035 - accuracy: 0.2419
Epoch 6/50
199/199 [==============================] - 14s 68ms/step - loss: 3.3558 - accuracy: 0.2928
Epoch 7/50
199/199 [==============================] - 14s 68ms/step - loss: 2.9781 - accuracy: 0.3464
Epoch 8/50
199/199 [==============================] - 14s 68ms/step - loss: 2.6672 - accuracy: 0.3939
Epoch 9/50
199/199 [==============================] - 14s 68ms/step - loss: 2.4066 - accuracy: 0.4409
Epoch 10/50
199/199 [==============================] - 14s 68ms/step - loss: 2.198

In [29]:
model.save("Text Generation.h5")

In [51]:
from google.colab import files

In [53]:
files.download("Text Generation.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Generating some text

In [36]:
seed_text= "I could not stop laughing at the ease with which explained his process of deduction"
next_words=100

for _ in range(next_words):
  sequence=tokenizer.texts_to_sequences([seed_text])
  padded=pad_sequences(sequence,max_sequence_len-1)
  predicted=model.predict(padded)
  predicted=np.argmax(predicted,axis=1)
  output_word=""
  for word, index in tokenizer.word_index.items():
    if index==predicted:
      output_word=word
     # print(output_word)
      break
  seed_text+=' '+output_word
print(seed_text)


I could not stop laughing at the ease with which explained his process of deduction to have plenty of playing these own police and i must be so frightened about half an hour colonel sherlock holmes was a man and was even that it was full of thirty a woman he was in a boiling fight against and every moment when the son was gone up right for him i see that she has said to reason from the business to the subject to mr jabez wilson here upon which you have read so little ” he picked it up so there and a few men in the lady’s lady’s room—you and a little methods
